# Types construits

En OCaml, on peut définir nos propres types.

## Type somme (variant)

On peut définir un **type somme (variant)** en énumérant tous les cas possibles pour avoir ce type. Ceci revient à prendre l'union de plusieurs types. Chaque cas doit être identifié par un nom appelé **constructeur**, qui doit commencer par une majuscule (il est interdit de commencer un nom de variable avec une majuscule, pour ne pas confondre) :

In [1]:
type matiere = Math | Physique | Info

type matiere = Math | Physique | Info


On vient de définir un **type** `matiere` qui contient trois **valeurs** Math, Physique, Info.

In [2]:
Math;;  (* c'est une valeur de type matiere *)
let a = Info;; (* qu'on peut stocker dans une variable *)

- : matiere = Math


val a : matiere = Info


Pour traiter une variable de ce type, on utilise un `match` :

In [3]:
let heures m = match m with
  | Math -> 12.
  | Physique -> 6.5
  | Info -> 4.;;
heures Info

val heures : matiere -> float = <fun>


Les constructeurs utilisés ci-dessus sont des constantes, mais il est possible d'ajouter un paramètre avec `of ...` :

In [4]:
type zbarre = Infini | MoinsInfini | Entier of int

type zbarre = Infini | MoinsInfini | Entier of int


Dans cet exemple, `zbarre` est censé représenter $\overline{\mathbb{Z}} = \mathbb{Z} \cup \{-\infty, \infty\}$. `Entier` est un constructeur qui dépend d'un paramètre entier. Pour obtenir une valeur à partir du constructeur `Entier`, on l'utilise comme une fonction :

In [5]:
Entier 10  (* valeur de type zbarre *)

- : zbarre = Entier 10


Écrivons une fonction pour augmenter une valeur de type `zbarre` de 1 :

In [6]:
let add1 n = match n with
  | Infini -> Infini    
  | MoinsInfini -> MoinsInfini
  | Entier i -> Entier (i + 1);;

add1 (Entier 10);;
add1 Infini

val add1 : zbarre -> zbarre = <fun>


- : zbarre = Entier 11


- : zbarre = Infini


Écrivons une fonction pour additionner deux valeurs de type `zbarre`. Pour pouvoir utiliser un opérateur infixe, on peut définir (+!) ([liste de tous les symboles autorisés comme opérateur infixe](https://caml.inria.fr/pub/docs/manual-caml-light/node4.9.html)):

In [7]:
let (+!) n1 n2 = match n1, n2 with
| Infini, Infini -> Infini
| MoinsInfini, MoinsInfini -> MoinsInfini
| Infini, Entier(_) | Entier(_), Infini  -> Infini
| MoinsInfini, Entier(_) | Entier(_), MoinsInfini  -> MoinsInfini
| _ -> failwith "Indetermine"

val ( +! ) : zbarre -> zbarre -> zbarre = <fun>


In [8]:
Infini +! Infini;;
Infini +! MoinsInfini

- : zbarre = Infini


error: runtime_error

**Exercice** : Définir `-`, `*`, `/` pour des valeurs de type `zbarre`. Définir aussi `~-` qui permet d'avoir d'appliquer - sur un seul élément (opérateur unaire).

Il est possible d'utiliser `'a` (n'importe quel type) pour un paramètre de constructeur, à condition de le mettre dans le `type`. Par exemple, on pourrait redéfinir `list` :

In [9]:
type 'a list = Vide | Cons of 'a * 'a list

type 'a list = Vide | Cons of 'a * 'a list


`Cons` est alors un constructeur ayant comme paramètre un couple : premier élément de la liste et reste de la liste. 

In [10]:
let l = Cons(1, Cons(2, Cons(3, Vide))) (* équivalent de 1::2::3::[]) *)

val l : int list = Cons (1, Cons (2, Cons (3, Vide)))


In [11]:
let rec appartient e l = match l with (* teste si e appartient à l *)
  | Vide -> false
  | Cons(x, q) -> x = e || appartient e q

val appartient : 'a -> 'a list -> bool = <fun>


In [12]:
type number = F of float | I of int

type number = F of float | I of int


In [13]:
let (+@) n1 n2 = match n1, n2 with
    | F(f1), F(f2) -> F(f1 +. f2)
    | I(n1), I(n2) -> I(n1 + n2)
    | I(n1), F(n2) -> F(float_of_int n1 +. n2)
    | F(n1), I(n2) -> F(n1 +. float_of_int n2)

val ( +@ ) : number -> number -> number = <fun>


In [14]:
let rec somme = function
    | [] -> I(0)
    | e::q -> e +@ somme q in
    
somme [F 3.14; I 2; I 98; F 1.745]

- : number = F 104.885


## Type enregistrement (record)

Alors qu'un type somme fait une disjonction (un "ou") de plusieurs types, un **type enregistrement (record)** permet d'avoir plusieurs types simultanément (un "et" de plusieurs types).

In [15]:
type fraction = {num: int; den: int};; (* fraction composée d'un numérateur ET dénominateur *)
let x = {num=3; den=4};; (* x représente la fraction 1/4 *)
x.den  (* obtient la valeur du champ den de x *)

type fraction = { num : int; den : int; }


val x : fraction = {num = 3; den = 4}


`fraction` est un type très proche de `int*int`. La différence principale est que les composantes sont nommées dans un enregistrement mais pas dans un tuple.  
Nous pouvons multiplier deux fractions :

In [16]:
let mult x1 x2 = 
  {num=x1.num*x2.num; den=x1.den*x2.den};;

  mult x x

val mult : fraction -> fraction -> fraction = <fun>


- : fraction = {num = 9; den = 16}


**Exercice** : Écrire une fonction pour additionner deux fractions.

**Exercice** : Écrire une fonction pour additionner deux fractions.

L'intérêt d'utiliser des fractions plutôt que des float est de faire des calculs exacts et non pas approchés.

### Tableau dynamique

In [17]:
type 'a dyn = {a: 'a array; size: int}

type 'a dyn = { a : 'a array; size : int; }


In [18]:
let d  = {a=[|1; 2; 3|]; size=3}

val d : int dyn = {a = [|1; 2; 3|]; size = 3}


In [19]:
d.a <- [|4; 5; 6|]

error: compile_error

In [20]:
let add d e = 
    if d.size = Array.length d.a 
    then let a' = Array.make (2*d.size) d.a.(0) in
    {a = a'; size = 2*d.size}
else (a.(size) <- e)

error: compile_error

In [21]:
type complexe = {re: float; im: float}
type polaire = {r: float; theta: float}

type complexe = { re : float; im : float; }


type polaire = { r : float; theta : float; }


In [22]:
let polaire_to_algebrique z =
    {re = z.r *. cos z.theta; im = z.r *. sin z.theta};;

polaire_to_algebrique {r = 1.; theta = 3.14/.2.}

val polaire_to_algebrique : polaire -> complexe = <fun>


- : complexe = {re = 0.000796326710733263345; im = 0.99999968293183461}
